# 중고나라 데이터 분석 프로젝트(가제)

## 합리적인 나이키 신발 구매

### 분석목적
- 가격 변화
- 중고가 ~ 현물가(머스트잇,크림) 비교
- 같은 제품의 플랫폼별(중고나라, 번개장터) 가격 비교 (강사님께 질문)
- 기간별 중고로 풀리는 명품 or 중고 수량 비교 (많이 풀리거나 떨어지는 기간이 있다면 인사이트가 될 것으로 기대)
- 네이버 검색량과 비교
- 허탕 비율 (내 검색어로 검색했는데 다른 상품이나왔을때) - 만약 이게 높다면 타사에 비해 경쟁력이 떨어지는 원인으로 인식될 수 있음
- 중고나라의 시세 추이가 현물 시세랑 얼마나 차이나는지/비슷한지
- 재발매일, 출시일에 대한 정보도 추가

### 데이터
- 날짜(일자별 데이터 from 중고나라)
- 등록 수 (from 중고나라)
- 가격 (중고나라 가격, 크림 가격-리셀, 정가)
- 상태(양호/보통/불량)
- 신발유형 (M/GS/PS)
- 검색어 트렌드 (from 네이버 데이터랩)


### 키워드
- 선정배경: 검색량 , 크림 판매수량 (예시 선정 기준)
- 나이키 덩크로우 범고래 + 나이키 덩크로우 블랙 (중고나라 :436건)
- 뉴발란스 992 그레이 (중고나라 : 121건)
- 커먼 프로젝트 아킬레스 로우 (중고나라 : 106건)
- 뉴발란스 327
- 마르지엘라 독일군 페인팅 (레드/블랙 제거, 103건)
- 피스마이너원 (지디 , 449건) : (여러 키워드로 검색해 데이터 추출가능)

<font color=red>[수정]
- 다시 정리</font>

## 패키지 임포트

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

import pandas as pd
import numpy as np


pd.options.display.max_rows=100
pd.options.display.max_columns=100

## 데이터 수집(크롤링)

- 번개장터 '키워드' 검색에 따른 상품 데이터

<font color=red>[추가]
1. 함수화시켜서 간단하게 적용할 수 있도록 만들어야 한다.
2. 키워드 for문으로 여러개 한번에 적용하도록 만든다.</font>

In [2]:
keyword = '나이키 덩크로우 범고래'

bunkae = pd.DataFrame()

info = {
    'referer': 'https://m.bunjang.co.kr/',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
    }

for i in range(0,10):
    
    # 차단막는 코드, 랜덤으로 time.sleep 지정
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # url 설정 (i는 0부터 시작, page=0,1,2,3 ....)
    url = f'https://api.bunjang.co.kr/api/1/find_v2.json?q={keyword}&order=score&page={i}&request_id=20211109155207&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4'

    print(f'{i}회차 크롤링 중입니다.')
    # requests로 데이터 요청하기
    resp = requests.get(url, headers = info)
    
    if resp.status_code == requests.codes.ok:
        data = json.loads(resp.text)
        prior_bunkae = pd.DataFrame(data['list'])
        bunkae = pd.concat([prior_bunkae, bunkae])

        
    else:
        print(f'{i}번째 요청이 잘 못 되었습니다.')
        pass # 넘겨준다.
              
print('크롤링 완료!!')

0회차 크롤링 중입니다.
1회차 크롤링 중입니다.
2회차 크롤링 중입니다.
3회차 크롤링 중입니다.
4회차 크롤링 중입니다.
5회차 크롤링 중입니다.
6회차 크롤링 중입니다.
7회차 크롤링 중입니다.
8회차 크롤링 중입니다.
9회차 크롤링 중입니다.
크롤링 완료!!


# 번개장터 데이터 설명

from : https://m.bunjang.co.kr/  (중고상품 판매 플랫폼 번개장터)

### 컬럼 설명

**굵은 글씨**는 우리 분석에 꼭 필요한 컬럼

- location : 등록 주소
- **name** : 판매자가 등록한 상품명
- **num_faved** : 좋아요 수
- pid : 상품아이디, @@@@@@@에 입력하면 해당 페이지로 이동함.-> https://m.bunjang.co.kr/products/@@@@@@@?q=%EB%82%98%EC%9D%B4%ED%82%A4%20%EB%8D%A9%ED%81%AC%EB%A1%9C%EC%9A%B0%20%EB%B2%94%EA%B3%A0%EB%9E%98&ref=%EA%B2%80%EC%83%89%EA%B2%B0%EA%B3%BC
- **price** : 가격
- product_image : 이미지 주소
- tag : 태그
<font color=red>- uid : 상품자 아이디? 아직 잘 모름
- status (0 / 1 / 3) : 이건 뭔지 아직 모름</font>
- **update_time** : 등록일 (<font color=red>기준은 찾아서 datetime형식으로 변환해야함</font>)
- **used** (1 = 중고 / 2 = 새상품 / 13 = 중고) : 상품상태  <font color=red>1과 13의 차이 아직 찾지 못했음</font>

# EDA

In [3]:
bunkae.head(1)

,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,num_comment,num_faved,only_neighborhood,outlink_url,pid,price,product_image,pu_id,ref_campaign,ref_code,ref_medium,ref_content,ref_source,status,style,super_up,tag,uid,update_time,used,bun_pay_filter_enabled,imp_id,ad_ref,faved
0,False,False,False,False,True,False,None,경기도 화성시 동탄1동,None,나이키 덩크로우 범고래 240,0,3,False,,167743900,300000,https://media.bunjang.co.kr/product/167743900_...,None,None,soldout_test_v2:B,None,,,3,,None,범고래 나이키범고래 범고래240,10416256,1634896799,1,False,383e618a61e1000ace60,,False
